In [1]:
import numpy as np #conda install -c anaconda numpy
import pandas as pd #conda install -c anaconda pandas
import seaborn as sns #conda install -c anaconda seaborn
from sqlalchemy import create_engine #conda install -c anaconda sqlalchemy
from sklearn.metrics.pairwise import cosine_similarity #conda install -c anaconda scikit-learn
from numpy import dot
from numpy.linalg import norm

In [2]:
host = 'localhost'  # localhost本地端
user = 'root'
password = ''
db_name = 'user_info'
port = '3306'  #3306

connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db_name}'
engine = create_engine(connection_string)

In [3]:
sql_query = 'SELECT rID,rName,all_label FROM res_info;'
res_info = pd.read_sql_query(sql_query, engine)

# res vector
dummies = res_info["all_label"].str.get_dummies(',')
res_vec = pd.concat([res_info["rID"], dummies], axis=1)
res_vec.set_index("rID",inplace=True)

print(res_info["all_label"].head())
print(res_vec.iloc[:5,:10])

0    中式,熱炒,合菜
1    粥,鍋物,鍋燒麵
2        飯,簡餐
3        中式,麵
4       咖啡,簡餐
Name: all_label, dtype: object
     三明治  三角餅  下午茶  中式  串燒  丼飯  乾麵  便當  健康餐  傻瓜麵
rID                                             
4      0    0    0   1   0   0   0   0    0    0
5      0    0    0   0   0   0   0   0    0    0
6      0    0    0   0   0   0   0   0    0    0
8      0    0    0   1   0   0   0   0    0    0
9      0    0    0   0   0   0   0   0    0    0


In [4]:
sql_query = 'SELECT uID,rID,rating FROM cost_detail;'
cost_detail = pd.read_sql_query(sql_query, engine)

In [5]:
#user vector 
res_rating = pd.merge(cost_detail[["uID","rID"]], res_vec, on='rID')
res_rating.drop(['rID'],axis=1,inplace=True)
user_vec = res_rating.groupby("uID").mean()

print(user_vec.iloc[:5,:10])

          三明治  三角餅  下午茶        中式   串燒   丼飯   乾麵   便當       健康餐  傻瓜麵
uID                                                                 
1    0.142857  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0
2    0.000000  0.0  0.0  0.285714  0.0  0.0  0.0  0.0  0.142857  0.0
3    0.000000  0.0  0.0  0.000000  0.0  0.0  0.0  0.0  0.000000  0.0
4    0.142857  0.0  0.0  0.142857  0.0  0.0  0.0  0.0  0.000000  0.0
5    0.100000  0.0  0.0  0.200000  0.0  0.0  0.0  0.1  0.200000  0.0


In [6]:
#用餘弦算相似度
user_res_similarity_matrix = cosine_similarity(user_vec.values,res_vec.values)
user_res_similarity_matrix = pd.DataFrame(user_res_similarity_matrix, index=user_vec.index,columns=res_vec.index)

num_user,num_res =  user_res_similarity_matrix.shape

In [7]:
print(f"user_res_similarity_matrix - #user: {num_user}, #res: {num_res}\n")
print(user_res_similarity_matrix.iloc[:5,:5])

user_res_similarity_matrix - #user: 20, #res: 720

rID         4    5         6         8         9
uID                                             
1    0.000000  0.0  0.188982  0.000000  0.566947
2    0.204124  0.0  0.125000  0.250000  0.375000
3    0.000000  0.0  0.258199  0.000000  0.516398
4    0.115470  0.0  0.141421  0.141421  0.282843
5    0.139010  0.0  0.340503  0.255377  0.425628


In [8]:
def get_the_most_similar_res(user_id, user_res_matrix,num):
    """Find the top-n restaurant most similar to the user"""
    user_vec = user_res_matrix.loc[user_id].values 
    sorted_index = np.argsort(user_vec)[::-1][:num]
    return list(user_res_matrix.columns[sorted_index])

In [9]:
#restaurant name
sql_query = 'SELECT rID,rName FROM restaurant;'
res_name = pd.read_sql_query(sql_query, engine)

In [11]:
#Find the top-10 restaurant most similar to the user1
user = 1
res_ids = get_the_most_similar_res(user, user_res_similarity_matrix,10)
print(f"user: {user}\n")
print(res_info[res_info["rID"].isin(res_ids)]["rName"])

user: 1

30        Bon Voyage Coffee 樂悠遊咖啡
35                  Feeling 18 咖啡
39               JULU Cafe（鉅鹿咖啡館）
40      Kètīng客廳（15:00-16:30休息時間）
48                       MuMu義式咖啡
49                 NU PASTA 南投埔里店
55                  Vesuvio 維蘇威咖啡
205                        名家咖啡总店
593    嫁婆jà po/露天咖啡請自備防蚊液小黑蚊沒在客氣的
654                         樂卓手搖飲
Name: rName, dtype: object


In [12]:
#Find the top-10 restaurant most similar to the user1
user = 2
res_ids = get_the_most_similar_res(user, user_res_similarity_matrix,10)
print(f"user: {user}\n")
print(res_info[res_info["rID"].isin(res_ids)]["rName"])

user: 2

30        Bon Voyage Coffee 樂悠遊咖啡
35                  Feeling 18 咖啡
39               JULU Cafe（鉅鹿咖啡館）
47           Mr.Drink手工飲品｜冷涼卡好果汁吧
55                  Vesuvio 維蘇威咖啡
56     YO COFFEE悠咖啡 （客製化蛋糕 埔里生日蛋糕
205                        名家咖啡总店
364                   姜有種 咖啡•酵母鬆餅
609                      聖多羅響味蔬活館
654                         樂卓手搖飲
Name: rName, dtype: object


In [13]:
#Find the top-10 restaurant most similar to the user1
user = 10
res_ids = get_the_most_similar_res(user, user_res_similarity_matrix,10)
print(f"user: {user}\n")
print(res_info[res_info["rID"].isin(res_ids)]["rName"])

user: 10

8                  272巷弄咖啡
48                MuMu義式咖啡
49          NU PASTA 南投埔里店
69     二月咖啡February coffee
193                   田都蔬食
238        江戶休閒舘（黑鼎coffee)
245                 西塔山咖啡坊
261                 甸畿咖啡莊園
321                  金埔里咖啡
411                   香緹咖啡
Name: rName, dtype: object
